Neste notebook importarei os dados do site basketball reference(https://www.basketball-reference.com/) para criar um dataset a partir de dados da  temporada regular da NBA desde a temporada 1984-1985(Quando Michael Jordan entrou como novato) até o fim da temporada 2020-2021
para analisar como seria a tabela de classificação da temporadas regulares de 2010-2011(Lebron entra no miami heat) até o fim da temporada 2017-2018(Lebron entra no Lakers),caso Michael Jordan entrasse na NBA na temporada 2010-2011

In [35]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
from urllib.error import URLError, HTTPError

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'}

times = {'Los Angeles Lakers' : 'LAL',
        'Milwaukee Bucks' : 'MIL',
        'Boston Celtics' : 'BOS',
        'Philadelphia 76ers' : 'PHI',
        'Portland Trail Blazers' : 'POR',
        'Detroit Pistons' : 'DET',
        'Denver Nuggets' : 'DEN',
        'Dallas Mavericks' : 'DAL',
        'Houston Rockets' : 'HOU',
        'San Antonio Spurs' : 'SAS',
        'New Jersey Nets' : 'NJN',
        'Utah Jazz' : 'UTA',
        'Golden State Warriors' : 'GSW',
        'Chicago Bulls' : 'CHI',
        'Cleveland Cavaliers' : 'CLE',
        'Indiana Pacers' : 'IND',
        'Phoenix Suns': 'PHO',
        'Los Angeles Clippers' : 'LAC',
        'Atlanta Hawks' : 'ATL',
        'Washington Bullets' : 'WSB',
        'New York Knicks' : 'NYK',
        'Seattle SuperSonics' : 'SEA',
        'Toronto Raptors' : 'TOR',
        'Miami Heat' : 'MIA',
        'Brooklyn Nets' : 'BRK',
        'Charlotte Hornets' : 'CHO',
        'Washington Wizards' : 'WAS',
        'Orlando Magic' : 'ORL'}

anos = ['2020']

todos_jogadores = []
todos_mvps = []

In [57]:
for ano in anos:
    "Nesta seção do código serão carregadas as páginas html"
    
    url_dados_time = f'https://www.basketball-reference.com/leagues/NBA_{ano}_standings.html'
    url_dados_basico_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_per_game.html'
    url_dados_avancados_jogador = f'https://www.basketball-reference.com/leagues/NBA_{ano}_advanced.html'
    url_votacao_mvp = f'https://www.basketball-reference.com/awards/awards_{ano}.html'
    
    "Aqui será carregado os dados dos candidatos a MVP relativos a este ano"
    try:
        req = Request(url_votacao_mvp, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    print(soup.find('tbody').findAll('tr'))
    
    try:
        req = Request(url_dados_time, headers = headers)

        response = urlopen(req)
        html = response.read()
        html = html.decode('utf-8')
        soup = BeautifulSoup(html, 'html.parser')

    except HTTPError as e:
        print(e.status, e.reason)

    except URLError as e:
        print(e.reason)
    
    "Aqui é a variável que armazenará os dados do time relativos àquele ano"
    dados_times = {}
    
    "Pegando os dados da Conferência Leste"
    
    times_html = soup.find('table', id = 'confs_standings_E').find('tbody').findAll('tr')

    cont = 1
    for time_html in times_html:
        dados_time = {}
        nome_time = time_html.find('th').find('a').getText()
        
        
        dados_time['conf_rank'] = cont
        cont+=1
        
        time_stats_html = time_html.findAll('td')
        
        for stat in time_stats_html:
            try:
                dados_time[stat.get('data-stat')] = float(stat.getText())
            except ValueError:
                dados_time[stat.get('data-stat')] = 0
    
        dados_times[nome_time] = dados_time
    
    "Pegando os dados da conferência Oeste"
    times_html = soup.find('table', id = 'confs_standings_W').find('tbody').findAll('tr')

    cont = 1
    for time_html in times_html:
        dados_time = {}
        nome_time = time_html.find('th').find('a').getText()
        
        
        dados_time['conf_rank'] = cont
        cont+=1
        
        time_stats_html = time_html.findAll('td')
        
        for stat in time_stats_html:
            try:
                dados_time[stat.get('data-stat')] = float(stat.getText())
            except ValueError:
                dados_time[stat.get('data-stat')] = 0
    
        dados_times[nome_time] = dados_time
    


[<tr><th class="right" data-stat="rank" scope="row">1</th><td class="left" csk="Antetokounmpo,Giannis" data-append-csv="antetgi01" data-stat="player"><a href="/players/a/antetgi01.html">Giannis Antetokounmpo</a></td><td class="right" data-stat="age">25</td><td class="left" data-stat="team_id"><a href="/teams/MIL/2020.html">MIL</a></td><td class="right" data-stat="votes_first">85.0</td><td class="right" data-stat="points_won">962.0</td><td class="right" data-stat="points_max">1010</td><td class="right" data-stat="award_share">0.952</td><td class="right" data-stat="g">63</td><td class="right" data-stat="mp_per_g">30.4</td><td class="right" data-stat="pts_per_g">29.5</td><td class="right" data-stat="trb_per_g">13.6</td><td class="right" data-stat="ast_per_g">5.6</td><td class="right" data-stat="stl_per_g">1.0</td><td class="right" data-stat="blk_per_g">1.0</td><td class="right" data-stat="fg_pct">.553</td><td class="right" data-stat="fg3_pct">.304</td><td class="right" data-stat="ft_pct">